In [2]:
import gym
import random



env = gym.make('Taxi-v3').env
env.render()

+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+



In [3]:
env.reset()
env.render()

+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+



In [4]:
print(env.action_space)

Discrete(6)


In [5]:
print(env.observation_space)

Discrete(500)


In [6]:
len(env.P)

500

In [8]:
env.P[265]

{0: [(1.0, 365, -1, False)],
 1: [(1.0, 165, -1, False)],
 2: [(1.0, 285, -1, False)],
 3: [(1.0, 245, -1, False)],
 4: [(1.0, 265, -10, False)],
 5: [(1.0, 265, -10, False)]}

In [17]:
import numpy as np

q_table = np.zeros([env.observation_space.n, env.action_space.n])

In [18]:
%%time

from IPython.display import clear_output

alpha = 0.1
gamma = 0.9
epsilon = 0.1

for i in range(1000000):
  estado = env.reset()

  penalidades, recompensa = 0, 0
  done = False
  while not done:
    if random.uniform(0, 1) < epsilon:
      acao = env.action_space.sample()
    else:
      acao = np.argmax(q_table[estado])

    proximo_estado, recompensa, done, info = env.step(acao)

    q_antigo = q_table[estado, acao]
    proximo_maximo = np.max(q_table[proximo_estado])

    q_novo = (1 - alpha) * q_antigo + alpha * (recompensa + gamma * proximo_maximo)
    q_table[estado, acao] = q_novo

    if recompensa == -10:
      penalidades += 1
    
    estado = proximo_estado
  
  if i % 100 == 0:
    clear_output(wait=True)
    print('Episódio: ', i)

print('Treinamento concluído!!!!')

Episódio:  999900
Treinamento concluído!!!!
CPU times: user 10min 52s, sys: 2min 13s, total: 13min 6s
Wall time: 11min 21s


In [21]:
q_table[223]

array([ -2.37440252,  -0.58568212,  -2.37440252,  -0.58568212,
       -10.52711391, -10.52711391])

In [24]:
env.reset()
env.render()

+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+



In [20]:
env.encode(2, 1, 0, 3)

223

### Avaliação


In [31]:
total_penalidades = 0
episodios = 50
frames = []

for _ in range(episodios):
  estado = env.reset()

  prenalidades, recompensa = 0, 0
  done = False
  while not done:
    acao = np.argmax(q_table[estado])
    estado, recompensa, done, info = env.step(acao)

    if recompensa == -10:
      penalidades += 1

    frames.append({
        'frame' : env.render(mode='ansi'),
        'state' : estado,
        'action' : acao,
        'reward' : recompensa
    })

  total_penalidades += penalidades

print('Episódios', episodios)
print('Penalidades', total_penalidades)

Episódios 50
Penalidades 150


In [38]:
from time import sleep

for frame in frames:
  clear_output(wait=True)
  print(frame['frame'])
  print('Estado', frame['state'])
  print('Ação', frame['action'])
  print('Recompensa', frame['reward'])
  sleep(.5)

+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (Dropoff)

Estado 410
Ação 5
Recompensa 20
